In [1]:
import sys
import os

In [2]:
!python --version

Python 3.9.16 :: Intel Corporation
 

# Installation Required one Time

In [3]:
!pip install langchain -q --user 

In [4]:
!pip install -U transformers -q --user 

In [5]:
!pip install pypdf -q --user

In [6]:
!pip install sentence-transformers -q --user

In [7]:
!pip install chromadb -q --user

In [8]:
!pip install google-cloud -q --user

In [11]:
!pip install modin -q --user

In [14]:
!pip install bitsandbytes -q --user

In [9]:
!pip install intel_extension_for_pytorch --user

# Imports

In [3]:
import time

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

In [5]:
from langchain.vectorstores import Chroma

In [6]:
from langchain.llms import HuggingFacePipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.schema.runnable import RunnablePassthrough

In [7]:
from langchain.chains import RetrievalQA

In [8]:
from chatbot_model import ChatBotModel

In [9]:
from input_data_loader import InputDataLoader

In [10]:
import modin.pandas as pd

2023-12-15 22:42:29,317 - numexpr.utils - INFO - Note: detected 224 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
2023-12-15 22:42:29,318 - numexpr.utils - INFO - Note: NumExpr detected 224 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-12-15 22:42:29,318 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [11]:
import pandas as default_pd

# DataLoader

In [12]:
chroma_db_dir = "1chroma-db"

# DATA Loader Core - Do 1 time

In [11]:
input_loader = InputDataLoader(persistent_db_dir=chroma_db_dir)
input_loader.core()

2023-12-15 12:24:13.310521: creating persistent database directory: 1chroma-db
2023-12-15 12:25:44.660243: Initializing SentenceTransformerEmbeddings


2023-12-15 12:25:45,668 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2023-12-15 12:25:46,679 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device: cpu


2023-12-15 12:25:46.680383: Creating a Chroma vector database from the text chunks


2023-12-15 12:25:46,910 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


Batches:   0%|          | 0/191 [00:00<?, ?it/s]

2023-12-15 12:27:03.024604: Making the Chroma vector database persistent
2023-12-15 12:27:03.024736: Total Time taken for Converting the Documents to Embeddings and store in the database: 169.71


# Define Model

In [13]:
chat_bot_model = ChatBotModel(
                 model_id_or_path = "mistralai/Mixtral-8x7B-Instruct-v0.1",
                 optimize=True
                    )

XPU or GPU not available - returning with CPU
2023-12-15 22:42:48.494552: torch_dtype : torch.bfloat16
2023-12-15 22:42:48.494572: device : cpu
2023-12-15 22:42:48.494575: model_id_or_path : mistralai/Mixtral-8x7B-Instruct-v0.1
2023-12-15 22:42:48.494582: AutoTokenizer.from_pretrained model_id_or_path : mistralai/Mixtral-8x7B-Instruct-v0.1 
2023-12-15 22:42:48.756070: Time Taken for tokenizer_init: 0.26 seconds
2023-12-15 22:42:48.756118: model_init: .AutoModelForCausalLM.from_pretrained model_id_or_path : mistralai/Mixtral-8x7B-Instruct-v0.1 


Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

2023-12-15 22:45:41.586142: Time Taken for model_init: 172.83 seconds
2023-12-15 22:45:41.586190: Enabling Intel ipex Optimizer


2023-12-15 22:46:25.689146: ChatBotModel Initialization Complete
2023-12-15 22:46:25.689255: Time Taken for model_init: 217.19 seconds


In [14]:
chat_bot_model_no_opt = ChatBotModel(model_id_or_path = "mistralai/Mixtral-8x7B-Instruct-v0.1",
                                     optimize=False)

XPU or GPU not available - returning with CPU
2023-12-15 22:47:59.877427: torch_dtype : torch.bfloat16
2023-12-15 22:47:59.877458: device : cpu
2023-12-15 22:47:59.877461: model_id_or_path : mistralai/Mixtral-8x7B-Instruct-v0.1
2023-12-15 22:47:59.877466: AutoTokenizer.from_pretrained model_id_or_path : mistralai/Mixtral-8x7B-Instruct-v0.1 
2023-12-15 22:48:00.025035: Time Taken for tokenizer_init: 0.15 seconds
2023-12-15 22:48:00.025055: model_init: .AutoModelForCausalLM.from_pretrained model_id_or_path : mistralai/Mixtral-8x7B-Instruct-v0.1 


Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

2023-12-15 22:48:36.763919: Time Taken for model_init: 36.74 seconds
2023-12-15 22:48:36.763977: Intel ipex Optimizer is not enabled as per request
2023-12-15 22:48:36.763982: ChatBotModel Initialization Complete
2023-12-15 22:48:36.763987: Time Taken for model_init: 36.89 seconds


In [15]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
# self.embeddings = SentenceTransformerEmbeddings(model_name="multi-qa-mpnet-base-dot-v1")

2023-12-15 22:48:48,832 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2023-12-15 22:48:49,983 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device: cpu


# Sample embedding output

In [21]:
text = "This is a test document."
query_result = embeddings.embed_query(text)
query_result[:3]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[-0.03833853453397751, 0.1234646737575531, -0.02864292822778225]

# Embed Documents

In [16]:
db = Chroma(persist_directory=chroma_db_dir, embedding_function=embeddings)

2023-12-15 22:49:01,082 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


# Create Pipeline

In [17]:
def _pipeline(max_length=512,temperature=0.3,top_p=0.95, bot_model=None):
    pipe = pipeline(
            'text2text-generation',
            model=bot_model.model,
            tokenizer=bot_model.tokenizer,
            max_length=max_length,
            do_sample=True,
            temperature=temperature,
            top_p=top_p
        )
    return HuggingFacePipeline(pipeline=pipe)

In [18]:
def _pipeline_mistral(max_length=512,temperature=0.3,top_p=0.95, bot_model=None):
    text_generation_pipeline = pipeline(
        model=bot_model.model,
        tokenizer=bot_model.tokenizer,
        task="text-generation",
        temperature=temperature,
        do_sample=True,
        repetition_penalty=1.1,
        return_full_text=True,
        max_new_tokens=1000,
    )
    return HuggingFacePipeline(pipeline=text_generation_pipeline)

In [19]:
llm_optimized = _pipeline_mistral(bot_model=chat_bot_model)
llm_not_optimized = _pipeline_mistral(bot_model=chat_bot_model_no_opt)

# Commented Section - Not Needed this was different Chain to Retrieve. 

In [31]:
# The model 'MixtralForCausalLM' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].

In [20]:
#  # Create a RetrievalQA chain
# qa_chain_optimized = RetrievalQA.from_chain_type(
#     llm=llm_optimized,
#     chain_type='stuff',
#     retriever=db.as_retriever(search_type="similarity", search_kwargs={"k": 2}),
#     return_source_documents=True,
# )

In [21]:

#  # Create a RetrievalQA chain
# qa_chain_not_optimized = RetrievalQA.from_chain_type(
#     llm=llm_not_optimized,
#     chain_type='stuff',
#     retriever=db.as_retriever(search_type="similarity", search_kwargs={"k": 2}),
#     return_source_documents=True,
# )

# Prompt Template

In [20]:
prompt_template = """
### [INST] 
Instruction: Assume You are expert in India's constitution and you know about India Law Sections, which is otherwise Indian Penal Code. 
Here is context to help:

{context}

### QUESTION:
{question} 

[/INST]
 """



In [21]:
# Create prompt from prompt template 
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)


# LLM Chains

In [22]:
# Create llm chain 
llm_chain_optimized = LLMChain(llm=llm_optimized, prompt=prompt)

In [23]:
# Create llm chain 
llm_chain_not_optimized = LLMChain(llm=llm_not_optimized, prompt=prompt)

In [24]:
# retriever = db.as_retriever()
retriever=db.as_retriever(search_type="similarity", search_kwargs={"k": 2})

In [26]:
def warm_up():
    query = "What is IPC" 
    rag_chain_optimized = ( 
     {"context": retriever, "question": RunnablePassthrough()}
        | llm_chain_optimized
    )
    
    resp = rag_chain_optimized.invoke(query)
    print(f"Response: {resp['text']}")

In [27]:
warm_up()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Response: 
The Indian Penal Code (IPC) is a comprehensive code of criminal law that came into operation on 1st January 1862. It applies throughout India except for the State of Jammu and Kashmir. The IPC provides for punishment for every act or omission that is contrary to its provisions, regardless of the nationality or rank of the person committing the offense. However, there are certain exceptions to this principle, such as the Sovereign and Foreign Sovereigns. Additionally, the IPC does not affect the provisions of any act for punishing mutiny and desertion by officers, soldiers, sailors, or airmen in the service of the Government of India.


# Rag Chain

In [29]:
rag_chain_not_optimized = ( 
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain_not_optimized
)
rag_chain_optimized = ( 
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain_optimized
)

In [30]:
def chat_bot_front_end_mistral_LLMChain():
    while True:
        # Prompt the user for a query
        input_query = str(input("Enter your query (or 'EXIT' to stop): "))
    
        # Check if the user entered 'EXIT' to stop the program
        if input_query.upper() == 'EXIT':
            print("Exiting...")
            break
        start_time = time.time()
        # Execute the query using the QA chain
        llm_response_not_optimized = rag_chain_not_optimized.invoke(input_query)
        inferencetime_not_optimized = round((time.time()-start_time), 2)
        # Print the response
        print(f"llm_response_not_optimized: {llm_response_not_optimized['text']}")
        print(f"inferencetime_not_optimized : {inferencetime_not_optimized}")
        
        print(f"Checking response with not optimized")
        start_time = time.time()
       
        # Execute the query using the QA chain
        llm_response_optimized = rag_chain_optimized.invoke(input_query)
        inferencetime_optimized = round((time.time()-start_time), 2)
        print(f"llm_response_optimized: {llm_response_optimized['text']}")
        print(f"inferencetime_optimized : {inferencetime_optimized}")
        

# Chat bot - Final

In [ ]:
chat_bot_front_end_mistral_LLMChain()

Enter your query (or 'EXIT' to stop):  Explain Article 5


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


llm_response_not_optimized: 
Article 5 of the Indian Constitution deals with the power of the Supreme Court to issue certain writs for the enforcement of fundamental rights. According to this article, the Supreme Court has the power to issue writs including habeas corpus, mandamus, prohibition, quo warranto, and certiorari for the enforcement of the fundamental rights conferred by Part III of the Constitution. These writs are issued to protect the citizens against the infringement of their fundamental rights by the State or its instrumentalities. The power of the Supreme Court under Article 5 is similar to the power of the High Courts under Article 226 of the Constitution. However, unlike the High Courts, the Supreme Court can issue these writs only for the enforcement of fundamental rights and not for any other purpose.

The document provided does not contain any information related to Article 5 directly. But, it gives some context regarding the powers of the Supreme Court and the jur

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


llm_response_not_optimized: 
Marriage laws in India are primarily governed by personal laws, which vary based on religion. However, the Indian Penal Code (IPC) contains provisions relating to certain offenses related to marriage, including bigamy.

Chapter XX of the IPC deals with offenses relating to marriage, and Section 494 specifically addresses the offense of marrying again during the lifetime of a husband or wife. According to this provision, whoever, having a husband or wife living, marries again during the lifetime of such husband or wife, shall be punished with imprisonment of either description for a term which may extend to seven years, and shall also be liable to fine.

However, there are exceptions to this rule. The section does not extend to any person whose marriage with such husband or wife has been declared void by a court of competent jurisdiction, nor to any person who contracts a marriage during the life of a former husband or wife, if such husband or wife, at the t

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


llm_response_optimized: 
Marriage laws in India are primarily governed by personal laws, which vary based on religion. However, the Indian Penal Code (IPC) contains provisions relating to certain offenses related to marriage, including bigamy.

Chapter XX of the IPC deals with offenses relating to marriage, and Section 494 of the IPC specifically addresses the offense of marrying again during the lifetime of the husband or wife. According to this section, whoever, having a husband or wife living, marries again in any case where such marriage is void by reason of its taking place during the life of such husband or wife, shall be punished with imprisonment of either description for a term which may extend to seven years, and shall also be liable to fine.

However, there are some exceptions to this rule. The section does not extend to any person whose marriage with such husband or wife has been declared void by a court of competent jurisdiction, nor to any person who contracts a marriage 

In [ ]:
chat_bot_front_end_mistral_LLMChain()

Enter your query (or 'EXIT' to stop):  Explain Article 5


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


llm_response_not_optimized: 
Article 5 of the Indian Constitution is not directly provided in the given page\_content of the documents. However, I can provide some information related to the articles and sections mentioned in the documents.

The first document mentions the words "under article 202," which refers to the jurisdiction and powers of the election commission. Article 202 provides for the power of the election commission to act judicially in certain matters. It states that while performing its duties, the election commission shall have the same powers as are vested in a civil court while trying a suit under the Code of Civil Procedure, 1908, in respect of the following matters:

* Enforcing the attendance of witnesses and examining them on oath;
* Discovery and production of documents;
* Receiving evidence on affidavits;
* Issuing commissions for the examination of witnesses or documents;

The second document talks about the commencement of the Constitution and its effect on 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


llm_response_not_optimized: 
According to the Indian Penal Code, a public servant is defined as a person who is in the service or pay of the Government and is entrusted with the performance of any public duty. This definition is not exhaustive, meaning that a person may be considered a public servant in terms of another statute. The term "public servant" cannot be extended to the provisions of the Representation of the People Act when this Act makes reference to persons in the service of the Government.

There are several offenses that can only be committed by public servants, and they also have privileges peculiar to themselves while discharging their duties.

The following individuals are considered public servants:

* Persons holding any office in virtue of which they are empowered to prepare, publish, maintain or revise an electoral roll or to conduct an election or part of an election.
* Every person who holds any office in virtue of which he is empowered to make, authenticate or 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


llm_response_optimized: 
In the Indian Penal Code, a "public servant" is defined as any person who is in the service or pay of the Government and is entrusted with the performance of any public duty. This definition is not exhaustive, and a person may be considered a public servant under another statute as well.

There are several offenses that can only be committed by public servants, and they also have certain privileges unique to their role. The term "public servant" includes those who hold an office that empowers them to prepare, publish, maintain, or revise an electoral roll or conduct an election. It also includes individuals who are in the service or pay of the Government, a local authority, a corporation established by or under a Central, Provincial, or State Act, or a Government company.

The definition of "public servant" cannot be extended to the provisions of the Representation of the People Act when it refers to persons in the service of the Government. However, wherever t

# Ignore Below

In [36]:
def chat_bot_front_end():
    while True:
        # Prompt the user for a query
        input_query = str(input("Enter your query (or 'EXIT' to stop): "))
    
        # Check if the user entered 'EXIT' to stop the program
        if input_query.upper() == 'EXIT':
            print("Exiting...")
            break
        start_time = time.time()
        # Execute the query using the QA chain
        llm_response_not_optimized = qa_chain_not_optimized({"query": input_query})
        inferencetime_not_optimized = round((time.time()-start_time), 2)
        # Print the response
        print(f"llm_response_not_optimized: {llm_response_not_optimized['result']}")
        print(f"inferencetime_not_optimized : {inferencetime_not_optimized}")
        
        start_time = time.time()
       
        # Execute the query using the QA chain
        llm_response_optimized = qa_chain_optimized({"query": input_query})
        inferencetime_optimized = round((time.time()-start_time), 2)
        print(f"llm_response_optimized: {llm_response_optimized['result']}")
        print(f"inferencetime_optimized : {inferencetime_optimized}")
        


In [35]:
chat_bot_front_end()

Enter your query (or 'EXIT' to stop):  What is IPC


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


llm_response_optimized:  IPC stands for Indian Penal Code. It was enacted in 1860 and came into operation on 1st January 1862. It applies throughout India except the state of Jammu and Kashmir. It makes every person liable to punishment for every act or omission contrary to its provisions, regardless of nationality or rank. However, it does not apply to the Sovereign or Foreign Sovereigns outside India. The IPC also does not affect the provisions of any act for punishing mutiny and desertion by officers, soldiers, sailors or airmen in the service of the Government of India.
inferencetime_optimized : 45.61


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


llm_response_not_optimized:  Section 320 of Indian Penal Code defines grievous hurt as follows:
(1) Emasculation;
(2) Permanent privation of the sight of either eye;
(3) Permanent privation of the hearing of either ear;
(4) Privation of any member or joint;
(5) Destruction or permanent impairing of the powers of any member or joint;
(6) Permanent disfiguration of head or face;
(7) Fracture or dislocation of a bone or tooth;
(8) Any hurt which endangers life or which causes the sufferer to be during the space of twenty days in severe bodily pain, or unable to follow his ordinary pursuits.
Explanation.—For the purposes of this section an “emasculation” means the removal of a testicle or penis.
inferencetime_not_optimized : 55.18


Enter your query (or 'EXIT' to stop):  EXIT


Exiting...


In [28]:
chat_bot_front_end()

Enter your query (or 'EXIT' to stop):  Article 5 explain


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


llm_response_optimized:  Article 5 of the Indian Constitution deals with the composition of the Rajya Sabha, which is one of the two Houses of Parliament in India. According to this article, the Rajya Sabha is composed of representatives of the States and of the Union territories. The total number of members of the Rajya Sabha is 245, out of which 233 are representatives of the States and 12 are representatives of the Union territories. Each State is represented in the Rajya Sabha by not more than ten members, and the representatives of each State are elected by the Legislative Assembly of that State in accordance with the system of proportional representation by means of the single transferable vote. The representatives of the Union territories are chosen in such manner as Parliament may by law provide.
inferencetime_optimized : 83.22


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


llm_response_not_optimized:  Contempt of court refers to actions or behaviors that disrespect, disrupt, or interfere with the authority and administration of justice in a court of law. This can include wilful disobedience to court orders, publishing material that scandalizes or lowers the authority of a court, or interfering with the due course of judicial proceedings. Contempt of court can be civil or criminal, and penalties may vary depending on the severity and nature of the offense.
inferencetime_not_optimized : 29.78


Enter your query (or 'EXIT' to stop):  EXIT


Exiting...


In [ ]:
chat_bot_front_end()

Enter your query (or 'EXIT' to stop):  Article 5 explain


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

llm_response_optimized: The context does not provide information about Article 5 explaining anything.
inferencetime_optimized : 0.69


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

llm_response_not_optimized: The context does not provide information about Article 5 explaining anything.
inferencetime_not_optimized : 0.71


...

Enter your query (or 'EXIT' to stop):  Define the word public


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

llm_response_optimized: The term used to describe something that is helpful to the public is "public nuisance".
inferencetime_optimized : 0.91


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

llm_response_not_optimized: The term used to describe something that is helpful is "public nuisance".
inferencetime_not_optimized : 1.09
